In [14]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import pyautogui
import time
import re

In [15]:
ops=webdriver.ChromeOptions()
ops.add_argument("--disable-notifications")

In [16]:
srv=Service("C:\py_selenium\chromedriver_win32\chromedriver.exe")
driver=webdriver.Chrome(service=srv,options=ops)

Incompatible release of chromedriver (version 113.0.5672.63) detected in PATH: C:\webdrivers\chromedriver.exe


In [17]:
driver.get("https://www.bbcgoodfood.com/")
driver.maximize_window()

In [18]:
# import time
# time.sleep(5)
# driver.find_element(By.XPATH,"//button[text()='AGREE']").click()

In [19]:
# alert=driver.switch_to.alert
# print(alert.text)
screen_width, screen_height = pyautogui.size()
x = int(screen_width * 0.6)  # Adjust as needed
y = int(screen_height * 0.75)  # Adjust as needed

pyautogui.moveTo(x, y)
pyautogui.click()
time.sleep(2)

In [20]:
from bs4 import BeautifulSoup
import requests

In [21]:
def func_time(ul_element):
    first_li = ul_element.find_all('li')[0]

    prep_time_ul = first_li.find('ul', class_='list')

    prep_ele = prep_time_ul.find('li')
    prep_text = prep_ele.get_text(separator=' ', strip=True) if prep_ele else None
    # print(prep_text)

    cook_ele = prep_ele.find_next_sibling('li')
    cook_text = cook_ele.get_text(separator=' ', strip=True) if cook_ele else None
    # print(cook_text)
    return prep_text, cook_text



def func_difficulty(ul_element):
    if ul_element.find('div', class_='post-header__skill-level'):
        return ul_element.find('div', class_='post-header__skill-level').get_text(strip=True)
    else:
        return None

    
def func_serves(ul_element):
    if ul_element.find('div', class_='post-header__servings'):
        return ul_element.find('div', class_='post-header__servings').get_text(strip=True)
    else:
        return None

def func_nutritional_vals(doc):
    nutrition_table = doc.find('table', class_="key-value-blocks")
    #print(nutrition_table.caption.string)

    tbodies = nutrition_table.find_all('tbody')
    nutrition_values = []

    for tbody in tbodies:
        for row in tbody.find_all('tr'):
            tdata = row.find_all('td')
            if len(tdata) >= 2:
                nutrition_values.append((tdata[1].get_text(strip=True), tdata[-1].get_text(strip=True)))
    #print(nutrition_values)
    return nutrition_values
                

def func_rating(doc):
    if not doc.find('div', class_='rating__values'):
        return None

    rating_string = doc.find('div', class_='rating__values').select('span.sr-only')[0].get_text(strip=True)
    rating_match = re.findall(r'\d+(?:\.\d+)?', rating_string)
    rating = float(rating_match[0]) if rating_match else None
    # print(rating)

    rating_count = doc.select('span.rating__count-text')[0].get_text(strip=True)
    # print(rating_count)
    return rating,rating_count

In [22]:
def func_scrape_dets(url):
    response = requests.get(url)
    doc = BeautifulSoup(response.content, "html.parser")
    title = doc.find(class_="heading-1")
    print(title.text)
    
    #func_scrape_dets(parent[0])
    
    ul_element = doc.find('ul', class_="post-header__planning")
    #ul_children = ul_element.children
    
    #li_elements = [child for child in ul_children if child.name == 'li']
    
    #getting prep and cook time
    time = func_time(ul_element)
    
    #getting difficulty
    difficulty = func_difficulty(ul_element)
    
    #getting serves
    serves = func_serves(ul_element)
    
    #getting nutritional values
    nutritions = func_nutritional_vals(doc)
    
    #getting ratings for popularity        
    rating = func_rating(doc)
    
    print(time)
    print(difficulty)
    print(serves)
    print(nutritions)
    print(rating)

In [23]:
# func_scrape_dets(parent[0])

In [24]:
func_scrape_dets("https://www.bbcgoodfood.com/recipes/chicken-chorizo-jambalaya")

Chicken & chorizo jambalaya
('Prep: 10 mins', 'Cook: 45 mins')
Easy
Serves 4
[('kcal', '445'), ('fat', '10g'), ('saturates', '3g'), ('carbs', '64g'), ('sugars', '7g'), ('fibre', '2g'), ('protein', '30g'), ('salt', '1.2g')]
(4.8, '2544 ratings')


In [25]:
ingredients=["chicken", "mushroom", "fish", "egg", "paneer", "beef", "soy bean"]

In [26]:
parent=[]

def func_getItems(driver):
    item=driver.find_elements(By.CLASS_NAME,"heading-4")
    item=item[1:len(item)-1]
    for i in range(len(item)):
        parent.append(item[i].find_element(By.XPATH,"./parent::a").get_attribute('href'))

In [27]:
for ingredient in ingredients:
    searchbox=driver.find_element(By.CLASS_NAME,"form-input")
    searchbox.clear()
    searchbox.send_keys(ingredient)
    searchbox.submit()
    func_getItems(driver)
    driver.back()

In [28]:
len(parent)

210

In [29]:
#their parent a tag which has the href
# parent=[]
# for i in range(len(item)):
#     parent.append(item[i].find_element(By.XPATH,"./parent::a").get_attribute('href'))

In [30]:
parent

['https://www.bbcgoodfood.com/recipes/chicken-chorizo-jambalaya',
 'https://www.bbcgoodfood.com/recipes/chicken-noodle-soup',
 'https://www.bbcgoodfood.com/recipes/chicken-pasta-bake',
 'https://www.bbcgoodfood.com/recipes/chicken-jalfrezi',
 'https://www.bbcgoodfood.com/recipes/chinese-chicken-curry',
 'https://www.bbcgoodfood.com/recipes/chicken-satay-salad',
 'https://www.bbcgoodfood.com/recipes/chicken-arrabbiata-stew-parmesan-dumplings',
 'https://www.bbcgoodfood.com/recipes/chicken-white-bean-stew',
 'https://www.bbcgoodfood.com/recipes/chicken-chorizo-ragu',
 'https://www.bbcgoodfood.com/recipes/summer-winter-chicken',
 'https://www.bbcgoodfood.com/recipes/mustard-stuffed-chicken',
 'https://www.bbcgoodfood.com/recipes/easy-chicken-curry',
 'https://www.bbcgoodfood.com/recipes/chicken-biryani',
 'https://www.bbcgoodfood.com/recipes/healthier-chicken-balti',
 'https://www.bbcgoodfood.com/recipes/home-style-chicken-curry',
 'https://www.bbcgoodfood.com/recipes/chicken-and-bean-enc

In [31]:
for i in parent:
    func_scrape_dets(i)

Chicken & chorizo jambalaya
('Prep: 10 mins', 'Cook: 45 mins')
Easy
Serves 4
[('kcal', '445'), ('fat', '10g'), ('saturates', '3g'), ('carbs', '64g'), ('sugars', '7g'), ('fibre', '2g'), ('protein', '30g'), ('salt', '1.2g')]
(4.8, '2544 ratings')
Chicken noodle soup
('Prep: 10 mins', 'Cook: 30 mins')
Easy
Serves 2
[('kcal', '217'), ('fat', '2g'), ('saturates', '0.4g'), ('carbs', '26g'), ('sugars', '1g'), ('fibre', '0.6g'), ('protein', '26g'), ('salt', '2.5g')]
(4.8, '751 ratings')
Chicken pasta bake
('Prep: 30 mins', 'Cook: 45 mins')
Easy
Serves 6
[('575', '575'), ('30g', '30g'), ('14g', '14g'), ('41g', '41g'), ('9g', '9g'), ('5g', '5g'), ('33g', '33g'), ('0.5g', '0.5g')]
(4.7, '590 ratings')
Chicken jalfrezi
('Prep: 10 mins', 'Cook: 1 hr')
Easy
Serves 4
[('kcal', '252'), ('fat', '8g'), ('saturates', '1g'), ('carbs', '11g'), ('sugars', '9g'), ('fibre', '5g'), ('protein', '30g'), ('salt', '0.4g')]
(4.5, '459 ratings')
Chinese chicken curry
('Prep: 15 mins', 'Cook: 40 mins')
Easy
Serves 4


Mushroom & spinach risotto
('Prep: 50 mins - 55 mins', None)
More effort
Serves 2
[('574', '574'), ('22g', '22g'), ('10g', '10g'), ('70g', '70g'), ('0g', '0g'), ('5g', '5g'), ('17g', '17g'), ('1.93g', '1.93g')]
(4.8, '278 ratings')
One pot chicken and mushroom risotto
('Prep: 10 mins', 'Cook: 55 mins')
Easy
Serves 4
[('629', '629'), ('22g', '22g'), ('12g', '12g'), ('62g', '62g'), ('4g', '4g'), ('4g', '4g'), ('37g', '37g'), ('2g', '2g')]
(4.8, '199 ratings')
Mushroom & potato curry
('Prep: 10 mins', 'Cook: 20 mins')
Easy
Serves 4
[('kcal', '212'), ('fat', '15g'), ('saturates', '9g'), ('carbs', '15g'), ('sugars', '5g'), ('fibre', '3g'), ('protein', '5g'), ('salt', '0.71g')]
(4.3, '371 ratings')
Bacon & mushroom pasta
('Prep: 5 mins', 'Cook: 20 mins')
Easy
Serves 4
[('kcal', '567'), ('fat', '20g'), ('saturates', '9g'), ('carbs', '78g'), ('sugars', '0g'), ('fibre', '4g'), ('protein', '23g'), ('salt', '1.41g')]
(4.7, '252 ratings')
Pan-fried chicken in mushroom sauce
('Total time 1 hr and 4

Beer-battered fish tacos
('Prep: 35 mins', 'Cook: 30 mins')
More effort
Serves 4
[('841', '841'), ('41g', '41g'), ('3g', '3g'), ('76g', '76g'), ('12g', '12g'), ('8g', '8g'), ('34g', '34g'), ('1.8g', '1.8g')]
(4.9, '26 ratings')
Coconut & kale fish curry
('Prep: 10 mins', 'Cook: 35 mins')
Easy
Serves 4
[('kcal', '465'), ('fat', '15g'), ('saturates', '6g'), ('carbs', '52g'), ('sugars', '7g'), ('fibre', '6g'), ('protein', '28g'), ('salt', '1.5g')]
(4.8, '31 ratings')
Harissa-crumbed fish with lentils & peppers
('Prep: 15 mins', 'Cook: 15 mins')
Easy
Serves 4
[('kcal', '425'), ('fat', '13g'), ('saturates', '1g'), ('carbs', '34g'), ('sugars', '2g'), ('fibre', '8g'), ('protein', '38g'), ('salt', '2g')]
(4.7, '54 ratings')
Super-quick fish curry
('Prep: 5 mins', 'Cook: 10 mins')
Easy
Serves 4
[('kcal', '191'), ('fat', '5g'), ('saturates', '1g'), ('carbs', '9g'), ('sugars', '6g'), ('fibre', '2g'), ('protein', '30g'), ('salt', '0.54g')]
(4.6, '161 ratings')
The best salmon fish cakes
('Prep: 20

Easter egg brownies
('Prep: 30 mins', 'Cook: 30 mins')
Easy
Serves 9 - 10
[('575', '575'), ('32g', '32g'), ('19g', '19g'), ('64g', '64g'), ('52g', '52g'), ('3g', '3g'), ('7g', '7g'), ('0.1g', '0.1g')]
(4.2, '49 ratings')
Avocado & black bean eggs
('Prep: 5 mins', 'Cook: 5 mins')
Easy
Serves 2
[('356', '356'), ('20g', '20g'), ('4g', '4g'), ('18g', '18g'), ('5g', '5g'), ('11g', '11g'), ('20g', '20g'), ('0.8g', '0.8g')]
(4.1, '47 ratings')
Easy scotch eggs
('Prep: 20 mins', 'Cook: 20 mins')
Easy
Makes 4
[('785', '785'), ('52g', '52g'), ('13g', '13g'), ('43g', '43g'), ('3g', '3g'), ('2g', '2g'), ('35g', '35g'), ('3.8g', '3.8g')]
(4.9, '34 ratings')
Egg & rocket pizzas
('Prep: 10 mins', 'Cook: 15 mins - 20 mins')
Easy
Serves 2
[('kcal', '327'), ('fat', '11g'), ('saturates', '3g'), ('carbs', '39g'), ('sugars', '8g'), ('fibre', '5g'), ('protein', '15g'), ('salt', '1g')]
(4.6, '38 ratings')
One-pan egg & veg brunch
('Prep: 5 mins', 'Cook: 25 mins')
Easy
Serves 2 adults + 2 children
[('170', '1

Paneer with broccoli & sesame
('Prep: 10 mins', 'Cook: 15 mins')
Easy
Serves 4
[('kcal', '195'), ('fat', '14g'), ('saturates', '6g'), ('carbs', '5g'), ('sugars', '3g'), ('fibre', '3g'), ('protein', '11g'), ('salt', '1.3g')]
(4.1, '13 ratings')
Tandoori paneer skewers with mango salsa
('Prep: 15 mins', 'Cook: 15 mins')
Easy
Serves 4 (makes 4 large or 8 small skewers
[('393', '393'), ('18g', '18g'), ('8g', '8g'), ('25g', '25g'), ('30g', '30g'), ('5g', '5g'), ('30g', '30g'), ('0.6g', '0.6g')]
(4.5, '6 ratings')
Indian paneer skewers
('Prep: 15 mins', 'Cook: 15 mins')
Easy
Serves 4
[('557', '557'), ('46g', '46g'), ('22g', '22g'), ('8g', '8g'), ('6g', '6g'), ('3g', '3g'), ('29g', '29g'), ('0.1g', '0.1g')]
(5.0, '2 ratings')
Paneer in herby tomato sauce
('Prep: 10 mins - 15 mins', 'Cook: 20 mins')
Easy
Serves 2
[('607', '607'), ('38g', '38g'), ('23g', '23g'), ('24g', '24g'), ('3g', '3g'), ('8g', '8g'), ('44g', '44g'), ('3.26g', '3.26g')]
(3.3, '12 ratings')
Herb & spice paneer fritters
('Pre

Easy beef in black bean sauce
('Prep: 15 mins', 'Cook: 15 mins')
Easy
Serves 4
[('297', '297'), ('12g', '12g'), ('3g', '3g'), ('15g', '15g'), ('10g', '10g'), ('4g', '4g'), ('30g', '30g'), ('1.38g', '1.38g')]
(4.7, '33 ratings')
Beef pho
('Prep: 25 mins', 'Cook: 45 mins')
Easy
Serves 2
[('kcal', '471'), ('fat', '18g'), ('saturates', '8g'), ('carbs', '44g'), ('sugars', '15g'), ('fibre', '6g'), ('protein', '29g'), ('salt', '3.83g')]
(4.8, '24 ratings')
Slow cooker beef curry
('Prep: 20 mins', 'Cook: 4 hrs and 30 mins - 8 hrs and 30 mins')
Easy
Serves 4 - 5
[('425', '425'), ('30g', '30g'), ('17g', '17g'), ('6g', '6g'), ('3g', '3g'), ('2g', '2g'), ('32g', '32g'), ('0.18g', '0.18g')]
(3.6, '88 ratings')
Puy lentil salad with soy beans, sugar snap peas & broccoli
('Prep: 10 mins', 'Cook: 15 mins')
Easy
Serves 4
[('kcal', '302'), ('fat', '7g'), ('saturates', '1g'), ('carbs', '42g'), ('sugars', '9g'), ('fibre', '8g'), ('protein', '22g'), ('salt', '1.41g')]
(4.8, '56 ratings')
Prawn, avocado & s

In [32]:
df1=[]

for url in parent:
    response = requests.get(url)
    doc = BeautifulSoup(response.content, "html.parser")
    title = doc.find(class_="heading-1")
    print(title.text)
    
    #func_scrape_dets(parent[0])
    
    ul_element = doc.find('ul', class_="post-header__planning")
    #ul_children = ul_element.children
    
    #li_elements = [child for child in ul_children if child.name == 'li']
    
    #getting prep and cook time
    time = func_time(ul_element)
    
    #getting difficulty
    difficulty = func_difficulty(ul_element)
    
    #getting serves
    serves = func_serves(ul_element)
    
    #getting nutritional values
    nutritions = func_nutritional_vals(doc)
    
    #getting ratings for popularity        
    rating = func_rating(doc)
    
    df1.append({'Item': title.text, 'Time': time, 'Difficulty': difficulty, 'Serves': serves, 'Nutritional Values': nutritions, 'Rating': rating})

Chicken & chorizo jambalaya
Chicken noodle soup
Chicken pasta bake
Chicken jalfrezi
Chinese chicken curry
Chicken satay salad
Chicken arrabbiata stew & parmesan dumplings
Easy chicken casserole
Chicken & chorizo ragu
Summer-in-winter chicken
Mustard-stuffed chicken
Easy chicken curry
Chicken biryani
Chicken balti
Home-style chicken curry
Chicken enchiladas
Easy teriyaki chicken
Chicken & chorizo paella
Creamy chicken stew
Moroccan chicken
Amatriciana chicken traybake
Coconut chicken curry
Chicken madras
Classic roast chicken & gravy
Easy chicken fajitas
Chicken souvlaki
Chicken parmigiana
Slow cooker chicken tikka masala
Chicken & bacon pasta
Slow-cooker chicken korma
Mushroom risotto
Mushroom stroganoff
Mushroom soup
Creamy mushroom pasta
Chicken and mushrooms
Chicken & mushroom puff pie
Mushroom & spinach risotto
One pot chicken and mushroom risotto
Mushroom & potato curry
Bacon & mushroom pasta
Pan-fried chicken in mushroom sauce
Chicken, kale & mushroom pot pie
Sausage & mushroom r

In [33]:
import pandas as pd
pd.DataFrame(df1)

,Item,Time,Difficulty,Serves,Nutritional Values,Rating
0,Chicken & chorizo jambalaya,"(Prep: 10 mins, Cook: 45 mins)",Easy,Serves 4,"[(kcal, 445), (fat, 10g), (saturates, 3g), (ca...","(4.8, 2544 ratings)"
1,Chicken noodle soup,"(Prep: 10 mins, Cook: 30 mins)",Easy,Serves 2,"[(kcal, 217), (fat, 2g), (saturates, 0.4g), (c...","(4.8, 751 ratings)"
2,Chicken pasta bake,"(Prep: 30 mins, Cook: 45 mins)",Easy,Serves 6,"[(575, 575), (30g, 30g), (14g, 14g), (41g, 41g...","(4.7, 590 ratings)"
3,Chicken jalfrezi,"(Prep: 10 mins, Cook: 1 hr)",Easy,Serves 4,"[(kcal, 252), (fat, 8g), (saturates, 1g), (car...","(4.5, 459 ratings)"
4,Chinese chicken curry,"(Prep: 15 mins, Cook: 40 mins)",Easy,Serves 4,"[(kcal, 264), (fat, 8g), (saturates, 1g), (car...","(4.7, 474 ratings)"
...,...,...,...,...,...,...
205,Sausage & white bean casserole,"(Prep: 20 mins, Cook: 1 hr and 5 mins)",Easy,Serves 4,"[(kcal, 363), (fat, 17g), (saturates, 6g), (ca...","(4.5, 100 ratings)"
206,Butter bean & chorizo stew,"(Prep: 5 mins, Cook: 15 mins)",Easy,Serves 4,"[(491, 491), (32g, 32g), (8g, 8g), (24g, 24g),...","(4.5, 148 ratings)"
207,Chipotle sweet potato & black bean stew with c...,"(Prep: 10 mins, Cook: 50 mins)",Easy,Serves 4,"[(547, 547), (23g, 23g), (12g, 12g), (60g, 60g...","(4.8, 132 ratings)"
208,Spanish meatball & butter bean stew,"(Prep: 15 mins, Cook: 35 mins)",Easy,Serves 3,"[(kcal, 435), (fat, 15g), (saturates, 5g), (ca...","(4.4, 219 ratings)"
